!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [26]:
import math
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/inmo/tide/data/emo/.cache/kobert_v1.zip
using cached model. /home/inmo/tide/data/emo/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/inmo/tide/data/emo/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

In [6]:
max_len = 512   # 텍스트 데이터 최대 길이
batch_size = 64

In [7]:
PATH = './models/'
model = torch.load(PATH+'10emotions_model_2_10epoch.pt', map_location='cpu')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + '10emotions_model_state_dict_2_10epoch.pt', map_location='cpu'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [8]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1050


In [9]:
emotion_list = ['분노', '악의', '슬픔', '절망', '당황', '불안', '열등', '상처', '사랑', '편안']

In [10]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, 512, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    model.eval()

    for (token_ids, valid_length, segment_ids, label) in test_dataloader:
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        # for i, e in zip(out[0], emotion_list):
        #     print(f'{e}: {round(float(i),4)}')
        return out

emotion_list = ['분노', '악의', '슬픔', '절망', '당황', '불안', '열등', '상처', '사랑', '편안']

In [ ]:
predict('우리의 믿음')

### 가사집 데이터 불러오기

In [64]:
from konlpy.tag import Okt, Kkma

lyrics_df = pd.read_csv('../song/song_data/csv/learn_song_lyrics.csv', index_col=0)
lyrics_df.head(5)

,song_id,title,genre,lyrics
0,52441,너에게로 또 다시,['발라드'],그 얼마나 오랜 시간을\n 짙은 어둠에서 서성거렸나\n 내 마음을 닫아 둔채로\n ...
1,53060,솔아 솔아 푸르른 솔아,['포크/블루스'],거센 바람이 불어와서 \n 어머님의 눈물이 \n 가슴속에 사무쳐 우는 \n 갈라진 ...
2,1017150,그 아픔까지 사랑한거야,['발라드'],너를 처음 만난 날 소리없이\n 밤새 눈은 내리고\n 끝도 없이 찾아드는 기다림\n...
3,53018,향기로운 추억 (응답하라 1988 삽입곡),['발라드'],한줌 젖은 바람은 \n 이젠 희미해진 옛 추억 \n 어느 거리로 \n 날 데리고 가...
4,1859404,잊지 말아요,['성인가요/트로트'],이젠 모두 지나버린 일이야 \n 사랑했던 그 추억 마저도 \n 하지만 멀리서 \n ...


In [66]:
lyrics_df[lyrics_df['song_id'] == 4226135]

,song_id,title,genre,lyrics
6412,4226135,좋아보여,"['발라드', '국내드라마']",다가오는 따뜻한 \n 온기가 가득하게 남아서 \n 긴 여행의 끝에 \n 집을 찾은 ...


### 문장별 리스트 및 한 줄 가사 생성

In [ ]:
title = '바람이 분다'

lyrics = lyrics_df[lyrics_df.title.str.contains(title)].head(1).lyrics.item()
lyrics_list = [l for l in lyrics.split(' \\n ') if l != '']
lyrics = lyrics.replace('\\n', '').replace('  ', ' ')

len(lyrics), lyrics

### 전체를 돌려버리기

In [ ]:
result = predict(lyrics)[0]
for n, e in zip(emotion_list, result):
    print(n, float(e))

### 형태소 분석을 통해 각 행마다의 값을 n으로 나누어 누적

In [ ]:
okt = Okt()
kkma = Kkma()

In [ ]:
res = predict(lyrics)[0]
pre_emo = [0]*10
s_len = len(kkma.sentences(lyrics))
for lyric in kkma.sentences(lyrics):
    emo_tensor = predict(lyric)
    for i in range(10):
        x = float(emo_tensor[0][i])/s_len
        pre_emo[i] += x
for i in range(10):
    res[i] -= round(pre_emo[i],4)

res

## 노래 가사의 감정 분석 결과

### 임의로 점수에 가중치를 더해 상위 3개의 감정만 추출

In [ ]:
def emo_rank(pre_result):
    if pre_result[8] >= math.sqrt(sum(abs(result))/10):
        pre_result[8] = max(float(pre_result[8] ** 2), pre_result[8])
    if pre_result[9] >= math.sqrt(sum(abs(result))/10):
        pre_result[9] = max(float(pre_result[9] ** 2), pre_result[9])
        
    for i in range(8):
        pre_result[i] = (pre_result[i]/5)*4
    
    mask = sorted(enumerate(pre_result), key=lambda x:x[1], reverse=True)
    e = []
    tmp = []
    for i, x in mask:
        if len(e) >= 3:
            break
        if x > float(sum(abs(result))/len(result)):
            e.append(i+1)
        elif x >= math.sqrt(abs(sum(result)))/len(result):
            tmp.append(i+11)
    else:
        while len(e) < 3:
            if tmp:
                e.append(tmp.pop(0))
            else:
                e.append(0)
    return e

In [ ]:
emo_rank_df = pd.DataFrame(columns=['song_id', 'title', '1emo','2emo','3emo'])

for i in tqdm(range(len(lyrics_df))):
    song_id = lyrics_df.loc[i, 'song_id']
    title = lyrics_df.loc[i, 'title']
    lyrics = lyrics_df.loc[i, 'lyrics'].replace('\\n', ' ')
    pre_res = predict(lyrics)
    e = emo_rank(pre_res[0])
    emo_rank_df.loc[i, :] = [song_id, title, e[0], e[1], e[2]]

emo_rank_df.to_csv('../song/lyrics_emotion_ver3.csv')

### 추출된 그대로, 소숫점 3자리까지 10개의 감정 모두 저장 (정수형으로 맞추기 위해 1000배)

In [ ]:
ten_emo_rating_df = pd.DataFrame(columns=['song_id', 'anger', 'malice', 'sad', 'dspair', 'panic', 'worry', 'complex', 'scar', 'love', 'happy'])

In [ ]:
for i in tqdm(range(len(lyrics_df))):
    lyrics = lyrics_df.loc[i, 'lyrics']
    lyrics = lyrics.replace('\\n', ' ').replace('  ', ' ').replace('  ', ' ')
    data = predict(lyrics)[0]
    
    rating = [int(float(i)*1000) for i in data]
    
    ten_emo_rating_df.loc[i, :] = [lyrics_df.loc[i, 'song_id'], *rating]

ten_emo_rating_df.to_csv('lyrics_10emotion_data.csv')

### 평균이 0이고 분산이 1인 노멀라이즈

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(ten_emo_rating_df.iloc[:, 1:11].values)

### 1순위 감정값 : 절대값의 평균 이상

In [ ]:
result < sum(abs(result))/10

### 2순위 감정값 : 절대값의 평균의 제곱근 이상

In [ ]:
result[(result > math.sqrt(sum(abs(result))/10)) & (result < sum(abs(result))/10)]

### 10개의 감정값 테이블과 새로운 row의 코사인 유사도 측정

In [13]:
emo = predict('나는 세상에서 프로그래밍이 제일 좋다. 싸피 최고! 취업도 성공할 수 있다!')[0]

In [20]:
df = pd.read_csv('./lyrics_10emotion_data.csv', index_col=0)

,song_id,anger,malice,sad,dspair,panic,worry,complex,scar,love,happy
1,52441,236,447,2406,-159,805,-757,779,2115,-2863,-2795
2,53060,383,-883,3310,2765,-870,820,738,250,-3326,-2907
3,1017150,-698,-578,4129,-622,-491,531,1336,319,-2370,-1475
4,53018,706,613,832,243,0,-1088,746,1648,-1625,-1937
5,1859404,1506,439,660,149,1102,1048,258,1070,-3256,-3154
...,...,...,...,...,...,...,...,...,...,...,...
9130,30117119,331,-925,3608,1571,297,544,406,778,-3385,-3076
9131,2795225,429,1466,1750,-233,1136,-827,749,1776,-2940,-2970
9132,469788,-293,-560,2679,-80,503,-197,-862,708,-895,-296
9133,8115185,964,107,488,-371,1537,741,-235,227,-2126,-1705


In [68]:
emo = predict('나는 싸피가 정말 좋다. 프로그래밍도 재미있어서 정말 다행이다. 꼭 대기업에 취직해야겠다!')

new_row = np.array([[int(i*1000) for i in emo.tolist()]])

new_row

array([[-1570, -1602, -1514, -1908,    64,  -593, -1543, -1608,  5465,
         6042]])

In [82]:
cons = cosine_similarity(df.iloc[:, 1:11].values, new_row)

In [100]:
sorted(enumerate(cons), reverse=True, key=lambda x: x[1])[:4]

[(430, array([0.98401915])),
 (8027, array([0.9700552])),
 (830, array([0.96526303])),
 (8321, array([0.96158195]))]

In [104]:
lyrics_df.iloc[8321, :]

song_id                                             31853557
title               오늘도 빛나는 너에게 (To You My Light) (Feat.이라온)
genre                                   ['R&B/Soul', '인디음악']
lyrics     별빛이 내린 밤 \n 그 풍경 속 너와 나 \n 날 새롭게 하는 \n 따뜻하게 만드...
Name: 8321, dtype: object

: 